In [55]:
import numpy as np
import pandas_datareader.data as web
import pandas as pd
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.value_at_risk import CVAROpt
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt.hierarchical_risk_parity import HRPOpt
from pypfopt.cla import CLA
import multiprocessing
from multiprocessing import Pool
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go

import ipywidgets as widgets

from math import pi
from bokeh.palettes import Category20c
from bokeh.transform import cumsum

from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import Range1d, HoverTool, CrosshairTool, NumeralTickFormatter
from bokeh.io import output_notebook
output_notebook()
#pip install PyPortfolioOpt

Loading BokehJS ...

# DATA RETRIEVE ##

In [84]:
import randomport
## 
def ParallelProcessPortfolioSim(data, numAssets, num_portfolios = 1000, risk_free_rate=0.0020):
    mu = expected_returns.mean_historical_return(data)
    S = risk_models.sample_cov(data)
    args = (mu, S, num_portfolios, risk_free_rate)

    if __name__ == '__main__':
        p = Pool(multiprocessing.cpu_count())
        results = p.starmap(randomport.display_simulated_ef_with_random, [(data, numAssets, mu, S, num_portfolios, risk_free_rate) for x in range(100)])
        results = pd.DataFrame(results)
        p.close()

    rp, sdp, rp_min, sdp_min = results[0].max(), results[1].min(), results[2].max(), results[3].min()
    print("-"*80)
    print("Maximum Sharpe Ratio Portfolio Allocation\n")
    print("Annualised Return:", round(rp,2))
    print("Annualised Volatility:", round(sdp,2))
    print("\n")
    #     print(results[4])
    print("-"*80)
    print("Minimum Volatility Portfolio Allocation\n")
    print("Annualised Return:", round(rp_min,2))
    print("Annualised Volatility:", round(sdp_min,2))
    print("\n")
#     (results[5])
    
## PLOTS SINGLE STOCK OVER TIME
def single_stock_performance(data,ticker):
    # Create figure
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=data.index,y=data[ticker].tolist()))
    # Set title
    fig.update_layout(title_text=ticker+' Price Movement')
    # Add range slider
    fig.update_layout(
        xaxis=go.layout.XAxis(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label="1m",
                         step="month",
                         stepmode="backward"),
                    dict(count=6,
                         label="6m",
                         step="month",
                         stepmode="backward"),
                    dict(count=1,
                         label="YTD",
                         step="year",
                         stepmode="todate"),
                    dict(count=1,
                         label="1y",
                         step="year",
                         stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )
    fig.show()

def cumulative_return(data):
    fig = figure(x_axis_type='datetime')
    growth = data / data.iloc[0] - 1.0
    source = ColumnDataSource(growth)
    colors=['blue','red','grey','green','orange']
    dic=dict(zip(stocks,colors))
    for key, value in dic.items():
        r = fig.line(x='Date',y=key,color=value, alpha=.25, line_width=1, source=source)
        hover = HoverTool(renderers=[r],tooltips=[(key, '(@Date{%F}, @' + key + ')')],formatters={'Date': 'datetime'}, toggleable=False)
        fig.add_tools(hover)
    fig.xaxis.axis_label='Time'
    fig.yaxis.axis_label='Return'
    fig.title.text = "5 Stock Cumulative Return"
    fig.title.align = "center"
    fig.title.text_font_size = "25px"
    
    return show(fig)

def portfolioPieChart(stocks,percent):
    colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen','lightblue']
    fig = go.Figure(data=[go.Pie(labels=stocks, 
                                 values=percent)])
    fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                      marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    fig.update_layout(title_text='Portfolio Weight',title_font_size=30)
    fig.show()

def multiStockPerformance(data,stocks):
    import plotly.graph_objects as go
    # Create figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data.index,y=data[stocks[0]].tolist(),name=stocks[0]))
    fig.add_trace(go.Scatter(x=data.index,y=data[stocks[1]].tolist(),name=stocks[1]))
    fig.add_trace(go.Scatter(x=data.index,y=data[stocks[2]].tolist(),name=stocks[2]))
    fig.add_trace(go.Scatter(x=data.index,y=data[stocks[3]].tolist(),name=stocks[3]))
    fig.add_trace(go.Scatter(x=data.index,y=data[stocks[4]].tolist(),name=stocks[4]))
    # Set title
    fig.update_layout(title_text=' Price Movement')
    # Add range slider
    fig.update_layout(
        xaxis=go.layout.XAxis(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label="1m",
                         step="month",
                         stepmode="backward"),
                    dict(count=6,
                         label="6m",
                         step="month",
                         stepmode="backward"),
                    dict(count=1,
                         label="YTD",
                         step="year",
                         stepmode="todate"),
                    dict(count=1,
                         label="1y",
                         step="year",
                         stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )
    #Add dropdown
    fig.update_layout(
    updatemenus=[
        go.layout.Updatemenu(
            active=0,
            buttons=list([
                dict(label=stocks[0],
                     method="update",
                     args=[{"visible": [True, False, False, False,False]},
                           {"title":stocks[0] }]),
                dict(label=stocks[1],
                     method="update",
                     args=[{"visible": [False, True, False, False,False]},
                           {"title": stocks[1]}]),
                dict(label=stocks[2],
                     method="update",
                     args=[{"visible": [False, False, True, False,False]},
                           {"title": stocks[2]}]),
                dict(label=stocks[3],
                     method="update",
                     args=[{"visible": [False, False, False, True,False]},
                           {"title": stocks[3]}]),
                dict(label=stocks[4],
                     method="update",
                     args=[{"visible": [False, False, False,False, True]},
                           {"title": stocks[4]}]),
            ]),
        )
    ])
    
    fig.show()
    
def stockUi(stock_list):
    '''UI for user to select the stock companies'''
    
    tab_contents = ['P0', 'P1', 'P2', 'P3', 'P4']
    children = [
    widgets.Dropdown(
        options=list(x for x in stock_list),
        value='MMM',
        description='Company:',
        disabled=False,
    )
     for name in tab_contents]
    tab = widgets.Tab()
    tab.children = children
    tab.children[0].value = "JPM"
    tab.children[1].value = "NKE"
    tab.children[2].value = "NFLX"
    tab.children[3].value = "AMZN"
    tab.children[4].value = "GOOGL"
    for i in range(len(children)):
        tab.set_title(i, "Stock" +str(i+1))
    return tab

def stockGenerate(tab):
    '''Function to genrate stock names'''
    User_Stocks=[]
    for i in range(5):
        User_Stocks.append(tab.children[i].value)
    return User_Stocks
def stockList500():
    sp_stocks = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    sp_stocks_tab= sp_stocks[0]
    sp_stocks_list= sp_stocks_tab['Symbol'].tolist()
    return sp_stocks_list


    #return df['JPM'].plot(label='JPM',figsize=(16,8),title='DESCRIPTIVE Plot'),df['NKE'].plot(label='NKE'),df['NFLX'].plot(label='NFLX'),df['AMZN'].plot(label='AMZN'),df['GOOGL'].plot(label='GOOGL')

# ------------------------------     COMPANY SELECTION       -------------------------------------

In [77]:
#ParallelProcessPortfolioSim(data, numAssets)

#
#portfolioPieChart(['arun','mario','arun2','mario1','arun3'],[0.3,0.3,0.2,0.1,0.1])
stock_list = stockList500()
tab=stockUi(stock_list)
print("SELECT YOUR STOCKS : ")
tab

SELECT YOUR STOCKS : 


# ------------------------------     DESCRIPTIVE STATISTICS   ------------------------------------

In [78]:
stocks = stockGenerate(tab)
numAssets = len(stocks)
source = 'yahoo'
start = '2010-01-01'
end = '2018-10-31'
data = pd.DataFrame()
for symbol in stocks:
        data[symbol] = web.DataReader(symbol, data_source=source, start=start, end=end)['Adj Close']

###  STOCK PRICE TREND 

In [92]:
multiStockPerformance(data,stocks)

### 5 POINT DESCRIPTION

In [93]:
data.describe()

,JPM,NKE,NFLX,AMZN,GOOGL
count,2225.000000,2225.000000,2225.000000,2225.000000,2225.000000
mean,54.477700,39.281574,88.633126,534.095407,583.959323
std,24.409543,18.149802,92.686185,452.082441,284.673251
min,22.752048,13.406624,7.018571,108.610001,218.253250
25%,34.196495,21.723955,24.014286,216.740005,308.558563
50%,49.363426,36.223881,54.432858,328.209991,545.320007
75%,60.922993,53.770588,115.209999,757.250000,785.789978
max,114.238289,84.623184,418.970001,2039.510010,1285.500000


In [79]:
cumulative_return(data)

# -------------------------- RANDOM PORTFOLIO GENERATIONS ----------------------------

In [80]:
ParallelProcessPortfolioSim(data, len(stocks))

--------------------------------------------------------------------------------
Maximum Sharpe Ratio Portfolio Allocation

Annualised Return: 88.99
Annualised Volatility: 3.56


--------------------------------------------------------------------------------
Minimum Volatility Portfolio Allocation

Annualised Return: 54.07
Annualised Volatility: 2.97


